In [1]:
# Импорт библиотек
import numpy as np
import pandas as pd
import lightgbm as lgb
import hyperopt
import random
import time
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score,mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold

In [2]:
X_train= pd.read_csv('pred_crim_train.csv', sep = ',', encoding = 'utf-8')
X_test= pd.read_csv('pred_crim_test.csv', sep = ',', encoding = 'utf-8')

In [3]:
X_train

,Id,Expected,НЕВЕРОТН.1..Невероятные.приключения.итальянцев.в.России,ТЕЛОХРАН.2..Телохранитель,КАВКАЗСК.3..Кавказская.пленница.или.Новые.приключения.Шурика,ПТЫЙ_ЭЛЕ.4..Пятый.элемент,ТЕРМИНАТ.5..Терминатор,КРЕПКИЙ.6..Крепкий.орешек,ПОЛИЦ_АК.7..Полицейская.академия,ЧУЖИЕ.8..Чужие,...,РУКА_ПЕР.191.Рука.переплетение.пальцев.рук...ведущая,РУКА_ИМИ.192.Рука.имитация.аплодисментов...ведущая,РУКА_СКР.193.Рука.скрещивание.рук.на.груди...ведущая,HОГА.194.Hога...ведущая,ГЛАЗ.195.Глаз...ведущая,РУКОЙ_ПИ.196.Рука.которой.пишут,ПЕРЕУЧИВ.197.Переучивали.ли.писать.если.левша,САМООПPЕ.198.Самоопpеделение.пpавша.левша,БОК_ПРЕД.199.Бок.на.котором.предпочитают.спать,ГЛАЗ_У_В.200.Глаз.который.обычно.шире.открыт
0,807,10.0,9.0,12.0,NaN,6.0,NaN,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,790,9.0,7.0,7.0,7.0,10.0,9.0,10.0,9.0,12.0,...,2.0,2.0,2.0,2.0,1.0,1.0,4.0,1.0,1.0,2.0
2,764,12.0,7.5,8.0,12.0,8.0,6.5,7.0,11.0,6.0,...,2.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,3.0,2.0
3,510,8.0,7.0,6.5,7.0,7.0,5.0,6.5,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1016,6.0,12.0,6.0,11.0,6.5,1.0,5.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2556,7.0,8.0,7.0,9.0,5.0,9.0,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,462,7.0,7.0,8.0,8.0,7.0,6.0,6.0,8.0,5.0,...,2.0,1.0,1.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0
997,304,10.0,7.0,8.0,7.0,8.0,8.0,10.0,11.0,7.0,...,1.0,1.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
998,762,8.0,6.0,5.0,7.0,9.0,7.0,7.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Сохраним индексы

In [4]:
Id=X_test.Id

In [5]:
del X_train['Id']
del X_test['Id']

In [6]:
X_train=X_train[X_train.Expected>0]


In [7]:
y=X_train.Expected
X_train=X_train.drop(['Expected'], axis = 1)

Посмотрим какие данные примерно должны получиться 

In [8]:
y.unique()

array([10. ,  9. , 12. ,  8. ,  6. ,  4. , 11. ,  7. ,  6.5,  9.1,  7.5,
        3. ,  5. ,  2. ,  9.7,  1. , 10.5,  9.9, 11.9,  9.6,  9.8, 10.9,
        8.7,  8.2, 11.5,  9.2,  4.7,  9.5,  8.8,  8.3])

In [9]:
X_train.columns


Index(['НЕВЕРОТН.1..Невероятные.приключения.итальянцев.в.России',
       'ТЕЛОХРАН.2..Телохранитель',
       'КАВКАЗСК.3..Кавказская.пленница.или.Новые.приключения.Шурика',
       'ПТЫЙ_ЭЛЕ.4..Пятый.элемент', 'ТЕРМИНАТ.5..Терминатор',
       'КРЕПКИЙ.6..Крепкий.орешек', 'ПОЛИЦ_АК.7..Полицейская.академия',
       'ЧУЖИЕ.8..Чужие', 'КРОКОДИЛ.9..Крокодил.Данди',
       'ЗДРАВСТВ.10..Здравствуйте.я.ваша.тетя.',
       ...
       'РУКА_ПЕР.191.Рука.переплетение.пальцев.рук...ведущая',
       'РУКА_ИМИ.192.Рука.имитация.аплодисментов...ведущая',
       'РУКА_СКР.193.Рука.скрещивание.рук.на.груди...ведущая',
       'HОГА.194.Hога...ведущая', 'ГЛАЗ.195.Глаз...ведущая',
       'РУКОЙ_ПИ.196.Рука.которой.пишут',
       'ПЕРЕУЧИВ.197.Переучивали.ли.писать.если.левша',
       'САМООПPЕ.198.Самоопpеделение.пpавша.левша',
       'БОК_ПРЕД.199.Бок.на.котором.предпочитают.спать',
       'ГЛАЗ_У_В.200.Глаз.который.обычно.шире.открыт'],
      dtype='object', length=161)

Заполняем пропуски

In [10]:
# imputer 
from sklearn.preprocessing import Imputer 
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0) 
imputer.fit(X_train)  
X_train = imputer.transform(X_train)


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
modelRF = RandomForestRegressor(n_estimators=100, max_features=1, random_state=1)
modelRF.fit(X_train, y)
score_train = modelRF.score(X_train, y)

In [13]:
parameters_rf = {
    "max_depth": [4,7,10],
    "min_samples_split": [2,3,4,10], 
    "min_samples_leaf": [2,3,4,10],
    "n_estimators": [30,60,90]
    #"criterion": ["gini", "entropy"]
}
clf_rf = GridSearchCV(RandomForestRegressor(random_state=1), parameters_rf, cv=6)

In [14]:
%%time
clf_rf.fit(X_train, y)
print(clf_rf.score(X_train, y))
print(clf_rf.best_params_)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.717427925246154
{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 90}
Wall time: 13min 47s


In [17]:
X_test

,НЕВЕРОТН.1..Невероятные.приключения.итальянцев.в.России,ТЕЛОХРАН.2..Телохранитель,КАВКАЗСК.3..Кавказская.пленница.или.Новые.приключения.Шурика,ПТЫЙ_ЭЛЕ.4..Пятый.элемент,ТЕРМИНАТ.5..Терминатор,КРЕПКИЙ.6..Крепкий.орешек,ПОЛИЦ_АК.7..Полицейская.академия,ЧУЖИЕ.8..Чужие,КРОКОДИЛ.9..Крокодил.Данди,ЗДРАВСТВ.10..Здравствуйте.я.ваша.тетя.,...,РУКА_ПЕР.191.Рука.переплетение.пальцев.рук...ведущая,РУКА_ИМИ.192.Рука.имитация.аплодисментов...ведущая,РУКА_СКР.193.Рука.скрещивание.рук.на.груди...ведущая,HОГА.194.Hога...ведущая,ГЛАЗ.195.Глаз...ведущая,РУКОЙ_ПИ.196.Рука.которой.пишут,ПЕРЕУЧИВ.197.Переучивали.ли.писать.если.левша,САМООПPЕ.198.Самоопpеделение.пpавша.левша,БОК_ПРЕД.199.Бок.на.котором.предпочитают.спать,ГЛАЗ_У_В.200.Глаз.который.обычно.шире.открыт
0,4.0,8.0,6.0,8.0,7.0,9.0,10.0,7.0,7.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,4.0,3.0,NaN,6.0,5.0,4.0,NaN,NaN,NaN,...,1.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
2,8.0,6.0,6.0,9.0,8.0,5.0,7.0,5.0,6.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.5,7.0,8.0,8.0,6.5,8.0,8.0,8.0,7.0,6.5,...,1.0,2.0,1.0,2.0,2.0,1.0,3.0,1.0,2.0,2.0
4,9.0,7.0,10.0,12.0,12.0,12.0,7.0,12.0,8.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,9.0,NaN,8.0,4.0,6.0,7.0,1.0,6.0,6.0,7.0,...,2.0,1.0,2.0,2.0,1.0,1.0,4.0,1.0,2.0,2.0
1195,6.0,NaN,6.0,7.0,6.0,7.0,6.0,NaN,7.0,NaN,...,1.0,2.0,2.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0
1196,9.0,8.0,9.0,9.0,7.0,NaN,4.0,NaN,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,NaN,NaN,7.0,NaN,8.0,11.0,5.0,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
X_test = imputer.transform(X_test)

In [19]:
pred_rf=clf_rf.predict(X_test)
pred_rf

array([8.3636837 , 9.3455816 , 8.68396555, ..., 7.30395003, 9.09250355,
       8.88007909])

In [21]:
pred=pd.DataFrame(Id,columns=['Id'])

In [22]:
pred['Predicted']=pred_rf

In [23]:
pred

,Id,Predicted
0,982,8.363684
1,3298,9.345582
2,112,8.683966
3,461,9.101718
4,907,10.118375
...,...,...
1194,1038,8.957197
1195,813,7.434495
1196,2141,7.303950
1197,3478,9.092504


In [24]:
pred.to_csv('regression_y.csv', index= False)